In [ ]:
import numpy as np
import math
import copy

# Massas molares g/mol
h2 = 2.016 
co = 28.01 
co2 = 44.01 
n2 = 28.0134 
ch3oh = 32.04
h2o = 18.01528

NE = 1 # número da equipe
cco = (53.5 + (NE * 0.25)) / 100 # conversão CO
A = (12.8 + NE * 0.2) # excesso H2

def molarmassa(C, el): # converte fração massica para molar 
    return(C / el)

 

In [ ]:
class Corrente: 
    pass

c = {} # correntes
for i in range(1, 24 + 1):
    c[i] = Corrente()
    
ch3ohC22 = 10000 # kg / h
c[22].ch3oh = molarmassa(ch3ohC22, ch3oh)
c[22].vm = c[22].ch3oh  / 0.999 # vm = vazão molar
c[22].h2o  = c[22].vm * 0.001

Reação Principal: CO + 2 H<sub>2</sub> &rarr; CH<sub>3</sub>OH <br>
Reação Secundária: CO<sub>2</sub> + 3 H<sub>2</sub> &rarr; CH<sub>3</sub>OH + H<sub>2</sub>O

# Abaixo Balanço sem reciclo

In [13]:
# BM sem reciclo
c[1].vm = 1000 # kmol vm = vazão molar
c[1].fco2 = 0.04 # f fração molar
c[1].fn2 = 0.01

while True: # variando c1 até o balanço fechar
    for c[1].fco in np.arange( 0.00001, 0.95, 0.00001): # variando a proporção h2 + co da entrada  
        c[1].fh2 = 0.95 - c[1].fco # restrição, reajustar como limite de loop talvez
        c[1].h2 = c[1].vm * c[1].fh2
        c[1].n2 = c[1].fn2 * c[1].vm
        # c6 reação 1
        ch3ohr1 = (c[1].fco * cco * c[1].vm) # ch3oh produzido
        cor1 = cco * (c[1].fco * c[1].vm) # co consumido 
        h2r1 = (c[1].fco * cco * c[1].vm * 2) # consumido h2
        h2r1est = (c[1].fco * c[1].vm * 2) # consumido se estequiometrico
        # c6 reação 2
        ch3ohr2 = (c[1].fco2 * c[1].vm) # produzido ch3oh
        h2or2 = (c[1].fco2 * c[1].vm) # produzido h2o
        h2r2 = 3 * (c[1].fco2 * c[1].vm) # consumido h2
        # c6
        c[6].h2 = c[1].vm * c[1].fh2 - (h2r1 + h2r2) # h2 em excesso que vai para c6
        c[6].ch3oh = ch3ohr1 + ch3ohr2 
        c[6].co = (c[1].fco * c[1].vm) - cor1
        c[6].n2 = c[1].n2
        c[6].h2o = h2or2
        c[6].vm = c[6].h2 + c[6].ch3oh + c[6].co + c[6].n2 + c[6].h2o
        # excesso h2 na reação 1  EXCESSO FELDER A = ((nA)_alimentação - (nA)_estequiometrico) /(nA)estequiometrico =13
        Ah2 = (c[1].h2 - h2r1est) / h2r1est
        if math.isclose(A, Ah2, rel_tol=1e-3):
            break

    # VS-1
    # c13
    c[13].ch3oh = 0.98 * c[6].ch3oh
    c[13].h2o =  0.96 * c[6].h2o
    c[13].fh2 = 0.001000
    c[13].fco = 0.000044
    c[13].fn2 = 0.000058
    c[13].fch3oh_h2o = 1 - (c[13].fco + c[13].fh2 + c[13].fn2) # fração combinada ch3oh e h2o
    c[13].vm = (c[13].ch3oh + c[13].h2o) / c[13].fch3oh_h2o
    c[13].n2 = c[13].vm * c[13].fn2
    c[13].h2 = c[13].vm * c[13].fh2
    c[13].co = c[13].vm * c[13].fco

    # c9 
    c[9].ch3oh =  c[6].ch3oh - c[13].ch3oh
    c[9].h2o =  c[6].h2o - c[13].h2o
    c[9].vm = c[6].vm - c[13].vm
    c[9].n2 = c[6].n2 - (c[13].n2)
    c[9].h2 = c[6].h2 - c[13].h2
    c[9].co = c[6].co - c[13].co
    c[9].fh2 = c[9].h2 / c[9].vm
    c[9].fco = c[9].co / c[9].vm
    c[9].fn2 = c[9].n2 / c[9].vm
    c[9].fch3oh = c[9].ch3oh / c[9].vm
    c[9].fh2o = c[9].h2o / c[9].vm
    # VS-2
    # c16
    c[16].ch3oh = 0.995 * c[13].ch3oh
    c[16].h2o =  0.98 * c[13].h2o
    c[16].fh2 = 0.000950
    c[16].fco = 0.000040
    c[16].fn2 = 0.000053
    c[16].fch3oh_h2o = 1 - (c[16].fco + c[16].fh2 + c[16].fn2) # fração combinada ch3oh e h2o
    c[16].vm = (c[16].ch3oh + c[16].h2o) / c[16].fch3oh_h2o
    c[16].n2 = c[16].vm * c[16].fn2
    c[16].co = c[16].vm * c[16].fco
    c[16].h2 = c[16].vm * c[16].fh2

    # c15
    c[15].ch3oh = c[13].ch3oh - c[16].ch3oh
    c[15].h2o =  c[13].h2o - c[16].h2o
    c[15].co = c[13].co - c[16].co
    c[15].h2 = c[13].h2 - c[16].h2
    c[15].vm = c[13].vm - c[16].vm
    c[15].n2 = c[13].n2 - (c[16].fn2 * c[16].vm)

    # DE-1 
    # c21
    c[21].vm = c[22].vm
    c[21].ch3oh = c[22].ch3oh
    c[21].h2o = c[22].h2o

    # c20
    c[20].vm = c[21].vm + c[22].vm
    c[20].ch3oh = c[22].ch3oh + c[21].ch3oh
    c[20].h2o = c[22].h2o + c[21].h2o 

    # c17
    c[17].n2 = c[16].n2
    c[17].co = c[16].co
    c[17].h2 = c[16].h2
    c[17].vm = c[17].n2 + c[17].co + c[17].h2 + c[20].vm
    c[17].ch3oh = c[20].ch3oh

    # c19
    c[19].n2 = c[17].n2
    c[19].co = c[17].co
    c[19].h2 = c[17].h2
    c[19].vm = c[19].n2 + c[19].co + c[19].h2

    # c24
    c[24].fh2o = 0.94
    c[24].fch3oh = 1 - c[24].fh2o
    c[24].h2o = c[16].h2o - c[22].h2o
    c[24].ch3oh = c[16].ch3oh - c[17].ch3oh
    c[24].vm = c[24].ch3oh + c[24].h2o

    if c[24].vm < 0 or c[24].ch3oh < 0: # caso contrario corrente fica com valor negativo, balanço parece fechar após qualquer valor acima d
        c[1].vm = c[1].vm + abs(c[24].ch3oh) + 0.1
    elif math.isclose(c[16].vm, c[17].vm + c[24].vm, rel_tol=1e-3):
        break
print("\nc24 ch3oh:",c[24].ch3oh, "\n c24:", c[24].vm, "\n c1", c[1].vm, "\n bm n2:", c[1].n2 - (c[9].n2 + c[15].n2 + c[19].n2))
print("\nfh2:", c[1].fh2, "\n c22 ch3oh:", c[22].ch3oh, "\n bm n2:", c[1].n2 - (c[9].n2 + c[15].n2 + c[19].n2), "\nbm ch3oh:", c[6].ch3oh-(c[9].ch3oh + c[15].ch3oh + c[22].ch3oh + c[24].ch3oh), "\n A:", Ah2)


c24 ch3oh: 0.0024237400708670975 
 c24: 417.9846705326146 
 c1 11115.39830669378 
 bm n2: 0.0

fh2: 0.9172699999999999 
 c22 ch3oh: 312.10986267166044 
 bm n2: 0.0 
bm ch3oh: 312.10986267166044 
 A: 13.012679498930643


In [ ]:
R = 0.0021111 
c[1].vm = 11113.39830669378
c[1].fco = 0.95 - 0.9173887 


In [ ]:

# c1
c[1].fh2 = 0.95 - c[1].fco # restrição
c[1].h2 = c[1].vm * c[1].fh2 
c[1].co = c[1].vm * c[1].fco
c[1].n2 = c[1].vm * c[1].fn2
c[1].co2 = c[1].vm * c[1].fco2
# c11
c[11].vm = c[9].vm * R 
c[11].fh2 = c[9].fh2
c[11].h2 = c[11].vm * c[11].fh2
c[11].fco = c[9].fco
c[11].co = c[11].vm * c[11].fco
c[11].fn2 = c[9].fn2
c[11].n2 = c[11].vm * c[11].fn2
c[11].fch3oh = c[9].fch3oh
c[11].ch3oh = c[11].vm * c[11].fch3oh
c[11].fh2o = c[9].fh2o
c[11].h2o = c[11].vm * c[11].fh2o
# c3
c[3].vm = c[1].vm + c[11].vm
c[3].co = c[1].co + c[11].co
c[3].co2 = c[1].co2 # co2 não está presente na c11
c[3].h2o = c[11].h2o # h2o não está presente na c1
c[3].h2 = c[1].h2 + c[11].h2
c[3].n2 = c[1].n2 + c[11].n2
c[3].ch3oh = c[11].ch3oh
# c6 reação 1 AJUSTAR PARA C3
ch3ohr1 = (c[3].co * cco)
cor1 = c[3].co * cco # consumido co
h2r1 = (c[3].co * cco * 2) # consumido h2
h2r1est = (c[3].co * 2) # consumido se estequiometrico
# c6 reação 2
ch3ohr2 = (c[3].co2)
h2or2 = (c[3].co2)
h2r2 = 3 * (c[3].co2)
# c6
c[6].h2 = c[3].h2 - (h2r1 + h2r2) # h2 em excesso que vai para c6
c[6].ch3oh = ch3ohr1 + ch3ohr2 + c[3].ch3oh
c[6].co = c[3].co - cor1
c[6].n2 = c[3].n2
c[6].h2o = h2or2 + c[3].h2o
c[6].vm = c[6].h2 + c[6].ch3oh + c[6].co + c[6].n2 + c[6].h2o
# excesso h2 na reação 1  EXCESSO FELDER A = ((nA)_alimentação - (nA)_estequiometrico) /(nA)estequiometrico =13
Ah2 = (c[3].h2 - h2r1est) / h2r1est

# VS-1
# c13
c[13].ch3oh = 0.98 * c[6].ch3oh
c[13].h2o =  0.96 * c[6].h2o
c[13].fh2 = 0.001000
c[13].fco = 0.000044
c[13].fn2 = 0.000058
c[13].fch3oh_h2o = 1 - (c[13].fco + c[13].fh2 + c[13].fn2) # fração combinada ch3oh e h2o
c[13].vm = (c[13].ch3oh + c[13].h2o) / c[13].fch3oh_h2o
c[13].n2 = c[13].vm * c[13].fn2
# c9 
c[9].ch3oh =  c[6].ch3oh - c[13].ch3oh
c[9].h2o =  c[6].h2o - c[13].h2o
c[9].vm = c[6].vm - c[13].vm
c[9].n2 = c[6].n2 - (c[13].n2)
c[9].h2 = c[6].h2 - c[13].h2
c[9].co = c[6].co - c[13].co
c[9].fh2 = c[9].h2 / c[9].vm
c[9].fco = c[9].co / c[9].vm
c[9].fn2 = c[9].n2 / c[9].vm
c[9].fch3oh = c[9].ch3oh / c[9].vm
c[9].fh2o = c[9].h2o / c[9].vm
# c11
c[11].vm = c[9].vm * R 
c[11].fh2 = c[9].fh2
c[11].h2 = c[11].vm * c[11].fh2
c[11].fco = c[9].fco
c[11].co = c[11].vm * c[11].fco
c[11].fn2 = c[9].fn2
c[11].n2 = c[11].vm * c[11].fn2
c[11].fch3oh = c[9].fch3oh
c[11].ch3oh = c[11].vm * c[11].fch3oh
c[11].fh2o = c[9].fh2o
c[11].h2o = c[11].vm * c[11].fh2o
# c10
c[10].vm = c[9].vm - c[11].vm 
c[10].fh2 = c[9].fh2
c[10].h2 = c[10].vm * c[10].fh2
c[10].fco = c[9].fco
c[10].co = c[10].vm * c[10].fco
c[10].fn2 = c[9].fn2
c[10].n2 = c[10].vm * c[10].fn2
c[10].fch3oh = c[9].fch3oh
c[10].ch3oh = c[10].vm * c[10].fch3oh
c[10].fh2o = c[9].fh2o
c[10].h2o = c[10].vm * c[10].fh2o
# R

# VS-2
# c16
c[16].ch3oh = 0.995 * c[13].ch3oh
c[16].h2o =  0.98 * c[13].h2o
c[16].fh2 = 0.000950
c[16].fco = 0.000040
c[16].fn2 = 0.000053
c[16].fch3oh_h2o = 1 - (c[16].fco + c[16].fh2 + c[16].fn2) # fração combinada ch3oh e h2o
c[16].vm = (c[16].ch3oh + c[16].h2o) / c[16].fch3oh_h2o
c[16].n2 = c[16].vm * c[16].fn2
c[16].co = c[16].vm * c[16].fco
c[16].h2 = c[16].vm * c[16].fh2
# c15
c[15].ch3oh = c[13].ch3oh - c[16].ch3oh
c[15].h2o =  c[13].h2o - c[16].h2o
c[15].co = c[13].co - c[16].co
c[15].h2 = c[13].h2 - c[16].h2
c[15].vm = c[13].vm - c[16].vm
c[15].n2 = c[13].n2 - (c[16].fn2 * c[16].vm)


# DE-1 
# c21
c[21].vm = c[22].vm

# c20
c[20].vm = c[21].vm + c[22].vm
c[20].ch3oh = c[22].ch3oh * 2
c[20].h2o = c[22].h2o * 2 

# c20
c[17].n2 = c[16].n2
c[17].co = c[16].co
c[17].h2 = c[16].h2
c[17].vm = c[17].n2 + c[17].co + c[17].h2 + c[20].vm
c[17].ch3oh = c[20].ch3oh

# c19
c[19].n2 = c[17].n2
c[19].co = c[17].co
c[19].h2 = c[17].h2
c[19].vm = c[19].n2 + c[19].co + c[19].h2

# c24
c[24].fh2o = 0.94
c[24].fch3oh = 1 - c[24].fh2o
c[24].h2o = c[16].h2o - c[22].h2o
c[24].ch3oh = c[16].ch3oh - c[17].ch3oh
c[24].vm = c[24].ch3oh + c[24].h2o

print("\nR:",R , "\nfh2:", c[1].fh2, "\n c1", c[1].vm, "\n bm n2:", c[1].n2 - (c[10].n2 + c[15].n2 + c[19].n2), "\nbm ch3oh:", c[6].ch3oh-(c[10].ch3oh + c[15].ch3oh + c[22].ch3oh + c[24].ch3oh), "\n A:", Ah2)


In [ ]:
print("\nn2 1:", c[9].vm ,"\nn2 3:", c[9].vm / R)


In [14]:
# BM com reciclo
# R = c[11].vm / c[9].vm

while True: # Rodar loop C1 { loop R {loop frac h2 {Parada Restrições todas batidas}} }
    for R in np.arange(0.001, 1, 0.001):
        for c[1].fco in np.arange( 0.001, 0.95, 0.001): # variando a proporção h2 + co da entrada
            # c1
            c[1].fh2 = 0.95 - c[1].fco # restrição
            c[1].h2 = c[1].vm * c[1].fh2 
            c[1].co = c[1].vm * c[1].fco
            c[1].n2 = c[1].vm * c[1].fn2
            c[1].co2 = c[1].vm * c[1].fco2
            # c11
            c[11].vm = c[9].vm * R 
            c[11].fh2 = c[9].fh2
            c[11].h2 = c[11].vm * c[11].fh2
            c[11].fco = c[9].fco
            c[11].co = c[11].vm * c[11].fco
            c[11].fn2 = c[9].fn2
            c[11].n2 = c[11].vm * c[11].fn2
            c[11].fch3oh = c[9].fch3oh
            c[11].ch3oh = c[11].vm * c[11].fch3oh
            c[11].fh2o = c[9].fh2o
            c[11].h2o = c[11].vm * c[11].fh2o
            # c3
            c[3].vm = c[1].vm + c[11].vm
            c[3].co = c[1].co + c[11].co
            c[3].co2 =c[1].co2 # co2 não está presente na c[11]
            c[3].h2o = c[11].h2o # h2o não está presente na c[11]
            c[3].h2 = c[1].h2 + c[11].h2
            c[3].n2 = c[1].n2 + c[11].n2
            c[3].ch3oh = c[11].ch3oh
            # c6 reação 1 AJUSTAR PARA C3
            ch3ohr1 = (c[3].co * cco)
            cor1 = (1 - cco) * (c[3].co) # sobra co
            h2r1 = (c[3].co * cco * 2) # consumido h2
            h2r1est = (c[3].co * 2) # consumido se estequiometrico
            # c6 reação 2
            ch3ohr2 = (c[3].co2)
            h2or2 = (c[3].co2)
            h2r2 = 3 * (c[3].co2)
            # c6
            c[6].h2 = c[3].h2 - (h2r1 + h2r2) # h2 em excesso que vai para c6
            c[6].ch3oh = ch3ohr1 + ch3ohr2 + c[3].ch3oh
            c[6].co = cor1
            c[6].n2 = c[3].n2
            c[6].h2o = h2or2 + c[3].h2o
            c[6].vm = c[6].h2 + c[6].ch3oh + c[6].co + c[6].n2 + c[6].h2o
            # excesso h2 na reação 1  EXCESSO FELDER A = ((nA)_alimentação - (nA)_estequiometrico) /(nA)estequiometrico =13
            Ah2 = (c[3].h2 - h2r1est) / h2r1est

            # VS-1
            # c13
            c[13].ch3oh = 0.98 *  c[6].ch3oh
            c[13].h2o =  0.96 * c[6].h2o
            c[13].fh2 = 0.001000
            c[13].fco = 0.000044
            c[13].fn2 = 0.000058
            c[13].fch3oh_h2o = 1 - (c[13].fco + c[13].fh2 + c[13].fn2) # fração combinada ch3oh e h2o
            c[13].vm = (c[13].ch3oh + c[13].h2o) / c[13].fch3oh_h2o
            c[13].n2 = c[13].vm * c[13].fn2
            c[13].h2 = c[13].vm * c[13].fh2
            c[13].co = c[13].vm * c[13].fco
            # c9 
            c[9].ch3oh =  c[6].ch3oh - c[13].ch3oh
            c[9].h2o =  c[6].h2o - c[13].h2o
            c[9].vm = c[6].vm - c[13].vm
            c[9].n2 = c[6].n2 - c[13].n2
            c[9].h2 = c[6].h2 - c[13].h2
            c[9].co = c[6].co - c[13].co
            c[9].fh2 = c[9].h2 / c[9].vm
            c[9].fco = c[9].co / c[9].vm
            c[9].fn2 = c[9].n2 / c[9].vm
            c[9].fch3oh = c[9].ch3oh / c[9].vm
            c[9].fh2o = c[9].h2o / c[9].vm
            # c11
            c[11].vm = c[9].vm * R 
            c[11].fh2 = c[9].fh2
            c[11].h2 = c[11].vm * c[11].fh2
            c[11].fco = c[9].fco
            c[11].co = c[11].vm * c[11].fco
            c[11].fn2 = c[9].fn2
            c[11].n2 = c[11].vm * c[11].fn2
            c[11].fch3oh = c[9].fch3oh
            c[11].ch3oh = c[11].vm * c[11].fch3oh
            c[11].fh2o = c[9].fh2o
            c[11].h2o = c[11].vm * c[11].fh2o
            # c10
            c[10].vm = c[9].vm -c[11].vm 
            c[10].fh2 = c[9].fh2
            c[10].h2 = c[10].vm * c[10].fh2
            c[10].fco = c[9].fco
            c[10].co = c[10].vm * c[10].fco
            c[10].fn2 = c[9].fn2
            c[10].n2 = c[10].vm * c[10].fn2
            c[10].fch3oh = c[9].fch3oh
            c[10].ch3oh = c[10].vm * c[10].fch3oh
            c[10].fh2o = c[9].fh2o
            c[10].h2o = c[10].vm * c[10].fh2o
            # R

            # VS-2
            # c16
            c[16].ch3oh = 0.995 * c[13].ch3oh
            c[16].h2o =  0.98 * c[13].h2o
            c[16].fh2 = 0.000950
            c[16].fco = 0.000040
            c[16].fn2 = 0.000053
            c[16].fch3oh_h2o = 1 - (c[16].fco + c[16].fh2 + c[16].fn2) # fração combinada ch3oh e h2o
            c[16].vm = (c[16].ch3oh + c[16].h2o) / c[16].fch3oh_h2o
            c[16].n2 = c[16].vm * c[16].fn2
            c[16].co = c[16].vm * c[16].fco
            c[16].h2 = c[16].vm * c[16].fh2
            # c15
            c[15].ch3oh = c[13].ch3oh - c[16].ch3oh
            c[15].h2o =  c[13].h2o - c[16].h2o
            c[15].co = c[13].co - c[16].co
            c[15].h2 = c[13].h2 - c[16].h2
            c[15].vm = c[13].vm - c[16].vm
            c[15].n2 = c[13].n2 - (c[16].fn2 * c[16].vm)


            # DE-1 
            # c21
            c[21].vm = c[22].vm

            # c20
            c[20].vm = c[21].vm + c[22].vm
            c[20].ch3oh = c[22].ch3oh * 2
            c[20].h2o = c[22].h2o * 2 

            # c20
            c[17].n2 = c[16].n2
            c[17].co = c[16].co
            c[17].h2 = c[16].h2
            c[17].vm = c[17].n2 + c[17].co + c[17].h2 + c[20].vm
            c[17].ch3oh = c[20].ch3oh

            # c19
            c[19].n2 = c[17].n2
            c[19].co = c[17].co
            c[19].h2 = c[17].h2
            c[19].vm = c[19].n2 + c[19].co + c[19].h2

            # c24
            c[24].fh2o = 0.94
            c[24].fch3oh = 1 - c[24].fh2o
            c[24].h2o = c[16].h2o - c[22].h2o
            c[24].ch3oh = c[16].ch3oh - c[17].ch3oh
            c[24].vm = c[24].ch3oh + c[24].h2o
            # ajustar condição!   
            if abs(Ah2 - A) < 0.1 and abs(c[1].n2 - (c[10].n2 + c[15].n2 + c[19].n2)) < 0.001:
                print("broke 1\n", c[1].fh2)
                break
            else:
                c[1].fh2 = 0.95 - c[1].fco # restrição, reajustar como limite de loop talvez
                c[1].h2 = c[1].vm * c[1].fh2
                c[1].n2 = c[1].fn2 * c[1].vm
                # c6 reação 1
                ch3ohr1 = (c[1].fco * cco * c[1].vm) # ch3oh produzido
                cor1 = (1 - cco) * (c[1].fco * c[1].vm) # co consumido 
                h2r1 = (c[1].fco * cco * c[1].vm * 2) # consumido h2
                h2r1est = (c[1].fco * c[1].vm * 2) # consumido se estequiometrico
                # c6 reação 2
                ch3ohr2 = (c[1].fco2 * c[1].vm) # produzido ch3oh
                h2or2 = (c[1].fco2 * c[1].vm) # produzido h2o
                h2r2 = 3 * (c[1].fco2 * c[1].vm) # consumido h2
                # c6
                c[6].h2 = c[1].vm * c[1].fh2 - (h2r1 + h2r2) # h2 em excesso que vai para c6
                c[6].ch3oh = ch3ohr1 + ch3ohr2 
                c[6].co = (c[1].fco * c[1].vm) - cor1
                c[6].n2 = c[1].fn2 * c[1].vm
                c[6].h2o = h2or2
                c[6].vm = c[6].h2 + c[6].ch3oh + c[6].co + c[6].n2 + c[6].h2o

                # VS-1
                # c13
                c[13].ch3oh = 0.98 *  c[6].ch3oh
                c[13].h2o =  0.96 * c[6].h2o
                c[13].fh2 = 0.001000
                c[13].fco = 0.000044
                c[13].fn2 = 0.000058
                c[13].fch3oh_h2o = 1 - (c[13].fco + c[13].fh2 + c[13].fn2) # fração combinada ch3oh e h2o
                c[13].vm = (c[13].ch3oh + c[13].h2o) / c[13].fch3oh_h2o
                c[13].n2 = c[13].vm * c[13].fn2
                c[13].h2 = c[13].vm * c[13].fh2
                c[13].co = c[13].vm * c[13].fco

                # c9 
                c[9].ch3oh =  c[6].ch3oh - c[13].ch3oh
                c[9].h2o =  c[6].h2o - c[13].h2o
                c[9].vm = c[6].vm - c[13].vm
                c[9].n2 = c[6].n2 - (c[13].fn2 * c[13].vm)
                c[9].h2 = c[6].h2 - c[13].h2
                c[9].co = c[6].co - c[13].co
                c[9].fh2 = c[9].h2 / c[9].vm
                c[9].fco = c[9].co / c[9].vm
                c[9].fn2 = c[9].n2 / c[9].vm
                c[9].fch3oh = c[9].ch3oh / c[9].vm
                c[9].fh2o = c[9].h2o / c[9].vm
                # VS-2
                # c16
                c[16].ch3oh = 0.995 * c[13].ch3oh
                c[16].h2o =  0.98 * c[13].h2o
                c[16].fh2 = 0.000950
                c[16].fco = 0.000040
                c[16].fn2 = 0.000053
                c[16].fch3oh_h2o = 1 - (c[16].fco + c[16].fh2 + c[16].fn2) # fração combinada ch3oh e h2o
                c[16].vm = (c[16].ch3oh + c[16].h2o) / c[16].fch3oh_h2o
                c[16].n2 = c[16].vm * c[16].fn2
                c[16].co = c[16].vm * c[16].fco
                c[16].h2 = c[16].vm * c[16].fh2

                # c15
                c[15].ch3oh = c[13].ch3oh - c[16].ch3oh
                c[15].h2o =  c[13].h2o - c[16].h2o
                c[15].co = c[13].co - c[16].co
                c[15].h2 = c[13].h2 - c[16].h2
                c[15].vm = c[13].vm - c[16].vm
                c[15].n2 = c[13].n2 - (c[16].fn2 * c[16].vm)

                # DE-1 
                # c21
                c[21].vm = c[22].vm
                c[21].ch3oh = c[22].ch3oh
                c[21].h2o = c[22].h2o

                # c20
                c[20].vm = c[21].vm + c[22].vm
                c[20].ch3oh = c[22].ch3oh + c[21].ch3oh
                c[20].h2o = c[22].h2o + c[21].h2o 

                # c17
                c[17].n2 = c[16].n2
                c[17].co = c[16].co
                c[17].h2 = c[16].h2
                c[17].vm = c[17].n2 + c[17].co + c[17].h2 + c[20].vm
                c[17].ch3oh = c[20].ch3oh

                # c19
                c[19].n2 = c[17].n2
                c[19].co = c[17].co
                c[19].h2 = c[17].h2
                c[19].vm = c[19].n2 + c[19].co + c[19].h2

                # c24
                c[24].fh2o = 0.94
                c[24].fch3oh = 1 - c[24].fh2o
                c[24].h2o = c[16].h2o - c[22].h2o
                c[24].ch3oh = c[16].ch3oh - c[17].ch3oh
                c[24].vm = c[24].ch3oh + c[24].h2o
        # ajustar condição!

        if abs(Ah2 - A) < 0.1 and abs(c[1].n2 - (c[10].n2 + c[15].n2 + c[19].n2)) < 0.001:
            print("broke 2\n")
            break
    # Aqui variar C1

    if abs(Ah2 - A) < 0.1 and abs(c[1].n2 - (c[10].n2 + c[15].n2 + c[19].n2)) < 0.001:
        break
    else:
        c[1].vm = c[1].vm - 1

print("\nR:",R , "\nfh2:", c[1].fh2, "\n c1", c[1].vm, "\n bm n2:", c[1].n2 - (c[10].n2 + c[15].n2 + c[19].n2), "\nbm ch3oh:", c[6].ch3oh-(c[10].ch3oh + c[15].ch3oh + c[22].ch3oh + c[24].ch3oh), "\n A:", Ah2)

broke 1
 0.9169999999999999
broke 2


R: 0.002 
fh2: 0.9169999999999999 
 c1 11115.39830669378 
 bm n2: 0.0004436619058196811 
bm ch3oh: 312.13554292029096 
 A: 12.902587034344963


In [15]:
print(c[24].ch3oh)

1.8005356471140885
